In the last lesson we encountered the Bitcoin protocol's [Version Handshake](https://en.bitcoin.it/wiki/Version_Handshake). We saw how Bitcoin network peers won't respond if you don't start the conversion with a `version` message.

But we cheated last lesson. I gave you a serialize `version` message and didn't tell you how I created it. And 

We were also lazy: we didn't parse the cryptic `payload` of the `version` message that our peer gave us.

We, too, we rude! After listening for our peer's `version` message we stopped listening and never received their `verack` message.

So you see, we have much to fix in this lesson!

To begin, I'm going to redefine everything from last lesson. I'm going to rename `Message` -> `Packet`.

In [12]:
from hashlib import sha256

NETWORK_MAGIC = 0xD9B4BEF9

def bytes_to_int(b):
    return int.from_bytes(b, 'little')

def read_magic(sock):
    magic_bytes = sock.recv(4)
    magic = bytes_to_int(magic_bytes)
    return magic

def read_command(sock):
    raw = sock.recv(12)
    # remove empty bytes
    command = raw.replace(b"\x00", b"")
    return command

def read_length(sock):
    raw = sock.recv(4)
    length = bytes_to_int(raw)
    return length

def read_checksum(sock):
    # FIXME: protocol documentation says this should be an integer ...
    raw = sock.recv(4)
    return raw

def calculate_checksum(payload_bytes):
    """First 4 bytes of sha256(sha256(payload))"""
    first_round = sha256(payload_bytes).digest()
    second_round = sha256(first_round).digest()
    first_four_bytes = second_round[:4]
    return first_four_bytes

def read_payload(sock, length):
    payload = sock.recv(length)
    return payload


class Packet:

    def __init__(self, command, payload):
        self.command = command
        self.payload = payload

    @classmethod
    def from_socket(cls, sock):
        magic = read_magic(sock)
        if magic != NETWORK_MAGIC:
            raise ValueError(f'Network magic "{magic}" is wrong')

        command = read_command(sock)
        payload_length = read_length(sock)
        checksum = read_checksum(sock)
        payload = read_payload(sock, payload_length)
        
        calculated_checksum = calculate_checksum(payload)
        if calculated_checksum != checksum:
            raise RuntimeError("Checksums don't match")

        if payload_length != len(payload):
            raise RuntimeError("Tried to read {payload_length} bytes, only received {len(payload)} bytes")

        return cls(command, payload)

    def to_bytes(self):
        pass

    def to_message(self):
        message_class = command_to_message_class(self.command)
        return message_class.from_payload(self.payload)

    def __repr__(self):
        return f"<Message command={self.command} payload={self.payload}>"

In [14]:
import socket

PEER_IP = "35.187.200.6"
PEER_PORT = 8333

# magic "version" bytestring
VERSION = b'\xf9\xbe\xb4\xd9version\x00\x00\x00\x00\x00j\x00\x00\x00\x9b"\x8b\x9e\x7f\x11\x01\x00\x0f\x04\x00\x00\x00\x00\x00\x00\x93AU[\x00\x00\x00\x00\x0f\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0f\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00rV\xc5C\x9b:\xea\x89\x14/some-cool-software/\x01\x00\x00\x00\x01'

sock = socket.socket()
sock.connect((PEER_IP, PEER_PORT))

# initiate the "version handshake"
sock.send(VERSION)

# receive their "version" response
version_message = Packet.from_socket(sock)

print(version_message.payload)

b'\x7f\x11\x01\x00\r\x04\x00\x00\x00\x00\x00\x003\xc2X[\x00\x00\x00\x00\x0f\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\r\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00{\xc5\xa7\x80\xa1\x87\xc1\xda\x10/Satoshi:0.16.0/\x8d$\x08\x00\x01'


Our next task is to parse this payload. I'll let you in on a secret: parsing the payload will work almost exactly the same as the `Packet.from_socket` method above. 

You will need to look interpret [this chart](https://en.bitcoin.it/wiki/Protocol_documentation#version)

![image](./images/version-message.png)

Here's an exercise: parse the version field

In [ ]:
def read_version(binary_stream):
    # read and interpret bytes from the stream
    

In [ ]:
import ipytest, pytest
import test_data


ipytest.clean_tests("test_read_version*")

def test_read_version_0():
    assert read_version(test_data.version_0) == 70015

def test_read_version_0():
    assert read_version(test_data.version_0) == 70015

def test_read_version_0():
    assert read_version(test_data.version_0) == 70015
    
ipytest.run_tests(doctest=True)


Now, here's a further question: Given a version message, tell me whether the node that sent it can send a `pong` message ([hint](https://bitcoin.org/en/developer-reference#protocol-versions)).

In [ ]:
class BaseMessage:

    def to_packet(self):
        return Packet(self.command, self.to_bytes())


class VersionMessage(BaseMessage):

    command = b"version"

    def __init__(self):
        pass

    @classmethod
    def from_payload(cls, payload):
        stream = io.BytesIO(payload)
        ...
        return cls(p1, p2, p3)

    def to_bytes(self):
        pass    